In [1]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Image settings
image_size = 128
batch_size = 64
num_classes = 5

# Paths
base_dir = 'splitted_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# CNN model

def create_cnn(filters, dropout_rate, learning_rate):
    model = Sequential([
        tf.keras.Input(shape=(image_size, image_size, 3)),

        Conv2D(filters, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(2, 2),

        Conv2D(filters * 2, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(2, 2),

        Conv2D(filters * 4, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Fitness function

def fitness_function(filters, dropout_rate, learning_rate):
    model = create_cnn(int(filters), float(dropout_rate), float(learning_rate))
    early_stop = EarlyStopping(patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(factor=0.5, patience=2)
    model.fit(train_generator,
              validation_data=val_generator,
              epochs=12,
              verbose=0,
              callbacks=[early_stop, reduce_lr])
    loss, accuracy = model.evaluate(val_generator, verbose=0)
    return accuracy

Found 210 images belonging to 5 classes.
Found 45 images belonging to 5 classes.
Found 45 images belonging to 5 classes.


In [2]:
# Particle class
class Particle:
    def __init__(self, bounds):
        self.position = [random.uniform(b[0], b[1]) for b in bounds]
        self.velocity = [0.0 for _ in bounds]
        self.best_position = self.position[:]
        self.best_score = -1

    def update_velocity(self, global_best, inertia=0.7, cognitive=1.2, social=1.8):
        for i in range(len(self.velocity)):
            r1, r2 = random.random(), random.random()
            cognitive_velocity = cognitive * r1 * (self.best_position[i] - self.position[i])
            social_velocity = social * r2 * (global_best[i] - self.position[i])
            self.velocity[i] = inertia * self.velocity[i] + cognitive_velocity + social_velocity

    def update_position(self, bounds):
        for i in range(len(self.position)):
            self.position[i] += self.velocity[i]
            self.position[i] = max(min(self.position[i], bounds[i][1]), bounds[i][0])

# PSO settings
bounds = [
    (48, 96),        # Filters
    (0.2, 0.4),      # Dropout
    (5e-4, 2e-3)     # Learning rate
]

num_particles = 12
max_iter = 15
particles = [Particle(bounds) for _ in range(num_particles)]
global_best = None
global_best_score = -1

for iteration in range(max_iter):
    print(f"Iteration {iteration + 1}/{max_iter}")
    for particle in particles:
        filters, dropout, lr = particle.position
        score = fitness_function(filters, dropout, lr)
        print(f"filters={int(filters)}, dropout={dropout:.2f}, lr={lr:.5f} --> val_acc={score:.4f}")

        if score > particle.best_score:
            particle.best_score = score
            particle.best_position = particle.position[:]

        if score > global_best_score:
            global_best_score = score
            global_best = particle.position[:]

    for particle in particles:
        particle.update_velocity(global_best)
        particle.update_position(bounds)

print("\nBest Hyperparameters Found:")
print(f"Filters: {int(global_best[0])}, Dropout: {global_best[1]:.2f}, Learning Rate: {global_best[2]:.5f}")
print(f"Best Validation Accuracy: {global_best_score:.4f}")

Iteration 1/15


2025-05-25 15:03:40.807341: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-05-25 15:03:40.807393: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-05-25 15:03:40.807398: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-05-25 15:03:40.807590: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-25 15:03:40.807613: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/simarkalsi/Cd Project Soil/soil/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `supe

filters=78, dropout=0.21, lr=0.00091 --> val_acc=0.2444
filters=58, dropout=0.35, lr=0.00152 --> val_acc=0.2000
filters=90, dropout=0.22, lr=0.00113 --> val_acc=0.2222
filters=49, dropout=0.24, lr=0.00126 --> val_acc=0.3778
filters=49, dropout=0.24, lr=0.00147 --> val_acc=0.2222
filters=74, dropout=0.24, lr=0.00138 --> val_acc=0.2222
filters=86, dropout=0.20, lr=0.00171 --> val_acc=0.4444
filters=81, dropout=0.27, lr=0.00073 --> val_acc=0.2444
filters=93, dropout=0.27, lr=0.00064 --> val_acc=0.1333
filters=52, dropout=0.37, lr=0.00141 --> val_acc=0.2222
filters=86, dropout=0.35, lr=0.00130 --> val_acc=0.2000
filters=94, dropout=0.28, lr=0.00133 --> val_acc=0.3778
Iteration 2/15
filters=86, dropout=0.20, lr=0.00129 --> val_acc=0.2000
filters=96, dropout=0.27, lr=0.00163 --> val_acc=0.4000
filters=84, dropout=0.21, lr=0.00119 --> val_acc=0.2000
filters=96, dropout=0.22, lr=0.00200 --> val_acc=0.4000
filters=79, dropout=0.20, lr=0.00187 --> val_acc=0.2889
filters=77, dropout=0.22, lr=0.00

In [3]:
# Final model
final_model = create_cnn(int(global_best[0]), float(global_best[1]), float(global_best[2]))
final_model.fit(train_generator,
                validation_data=val_generator,
                epochs=20,
                callbacks=[EarlyStopping(patience=5, restore_best_weights=True),
                           ReduceLROnPlateau(factor=0.5, patience=2)],
                verbose=1)

loss, test_accuracy = final_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.2979 - loss: 155.6301 - val_accuracy: 0.2000 - val_loss: 210.9597 - learning_rate: 0.0017
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 792ms/step - accuracy: 0.5515 - loss: 140.4032 - val_accuracy: 0.2667 - val_loss: 121.0663 - learning_rate: 0.0017
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 657ms/step - accuracy: 0.5776 - loss: 110.4458 - val_accuracy: 0.3778 - val_loss: 46.5087 - learning_rate: 0.0017
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 648ms/step - accuracy: 0.5025 - loss: 96.8934 - val_accuracy: 0.2000 - val_loss: 92.4308 - learning_rate: 0.0017
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 646ms/step - accuracy: 0.5480 - loss: 83.1912 - val_accuracy: 0.2000 - val_loss: 42.2730 - learning_rate: 0.0017
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 629ms/step - accuracy: 0.6074 - loss: 57.9583 - val_accuracy: 0.3333 - val_loss: 10.3224 - learning_rate: 0.0017
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 624ms/step - accuracy: 0.5442 - loss: 93.2412 -